In [3]:
import pandas as pd
import itertools

In [4]:
df = pd.read_csv("C:/D drive/PSG/sem8/20XW87-DATA-MINING-LAB/ca1-practice/KaggleCovidDataset-Copy1.csv").applymap(lambda x:1 if x=="Yes" else 0)

In [5]:
df

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,1,1,1,1,1,0,0,0,0,1,...,1,1,0,1,0,1,1,0,0,1
1,1,1,1,1,0,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1,1,0,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
5430,1,1,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
5431,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5432,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def apriori_algorithm(df, min_support=0.1):
    
    df_copy = df.copy(deep=True)
    
    f_one_itemsets = {}
    
    for col in df.columns:
        item_count = df[col].sum()/df.shape[0]
        if item_count>min_support:
            f_one_itemsets[frozenset((col,))] = item_count
            
    def generate_k_frequent_itemsets(frequent_itemset, k):
        frequent_k_itemsets = set()
        
        for i1, i2 in itertools.combinations(frequent_itemset, 2):
            union_set = i1.union(i2)
            
            if len(union_set)==k:
                subsets = [frozenset(c) for c in itertools.combinations(union_set, k-1)]
                
                if all([subset in frequent_itemset for subset in subsets]):
                    frequent_k_itemsets.add(union_set)
                
        frequent_k_itemsets_support = {}
        
        for k_itemset in frequent_k_itemsets:
            support = df_copy[list(k_itemset)].all(axis=1).mean()
            
            if support>=min_support:
                frequent_k_itemsets_support[k_itemset] = support
                
        return frequent_k_itemsets_support

    all_frequent_itemsets = {**f_one_itemsets}
    frequent_k_itemsets = f_one_itemsets
    k = 2
    while len(frequent_k_itemsets)>0:
        frequent_k_itemsets = generate_k_frequent_itemsets(set(frequent_k_itemsets.keys()), k)
        all_frequent_itemsets.update(frequent_k_itemsets)
        k += 1
    
    return pd.DataFrame([(support, itemset) for itemset, support in all_frequent_itemsets.items()], columns=['support', 'itemset'])

In [29]:
def association_rules(df, min_cf=0.5):
    
    frequent_itemsets_map = {frozenset(itemset): support for support, itemset in zip(df['support'], df['itemset'])}
    associations = []
    for itemset in frequent_itemsets_map:
        
        if len(itemset)>1:
            for i in range(1, len(itemset)):
                antecendants_itemset = list(itertools.combinations(itemset, i))
                
                for antecedant in antecendants_itemset:
                    antecedant = frozenset(antecedant)
                    consequent = antecedant.difference(itemset)
                    
                    conf = frequent_itemsets_map[itemset]/frequent_itemsets_map[antecedant]
                    
                    if conf>=min_cf:
                        associations.append((antecedant, consequent, conf))
                        
                        
    return pd.DataFrame(associations, columns=['antecedants', 'consequents', 'confidence'])


In [16]:
fdf = apriori_algorithm(df)

In [30]:
association_rules(fdf, 0.6)

,antecedants,consequents,confidence
0,(Breathing Problem),(),0.838398
1,(Dry Cough),(),0.704667
2,(Chronic Lung Disease),(),0.773879
3,(Contact with COVID Patient),(),0.853632
4,(Asthma),(),0.704455
...,...,...,...
24362,"(Breathing Problem, Fever, Attended Large Gath...",(),0.669789
24363,"(Breathing Problem, Fever, Attended Large Gath...",(),1.000000
24364,"(Breathing Problem, Fever, COVID-19, Visited P...",(),0.633444
24365,"(Breathing Problem, Attended Large Gathering, ...",(),0.928571
